
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# Batch Deployment

Batch inference is the most common way of deploying machine learning models.  This lesson introduces various strategies for deploying models using batch including Spark. In addition, we will show how to enable optimizations for Delta tables.

**Learning Objectives:**

*By the end of this demo, you will be able to;*

* Load a logged Model Registry model using `pyfunc`.

* Compute predictions using `pyfunc` APIs.

* Perform batch inference using Feature Engineering's `score_batch` method.

* Materialize predictions into inference tables (Delta Lake).

* Perform common write optimizations like liquid clustering, predictive optimization to maximize data skipping and on inference tables.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**

**🚨 Prerequisites:** 
* **Feature Engineering** and **Feature Store** are not focus of this lesson. This course expect that you already know these topics. If not, you can check the **Data Preparation for Machine Learning** course.

* Model development with MLFlow is not in the scope of this course. If you need to refresh your knowledge about model tracking and logging, you can check the **Machine Learning Model Development** course.


## Classroom Setup

Before starting the demo, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%run ../Includes/Classroom-Setup-01

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| dropping the catalog "labuser8891327_1736105957_wp0t_da"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01"

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)
Creating & using the catalog "labuser8891327_1736105957_wp0t_da"...(2 seconds)

Predefined tables in "labuser8891327_1736105957_wp0t_da.default":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser8891327_1736105957@vocareum.com/machine-learning-model-deployment
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01

Setup completed (9 seconds)


**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"User DB Location:  {DA.paths.datasets}")

Username:          labuser8891327_1736105957@vocareum.com
Catalog Name:      labuser8891327_1736105957_wp0t_da
Schema Name:       default
Working Directory: dbfs:/mnt/dbacademy-users/labuser8891327_1736105957@vocareum.com/machine-learning-model-deployment
User DB Location:  dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01


## Data Preparation

For this demonstration, we will utilize a fictional dataset from a Telecom Company, which includes customer information. This dataset encompasses **customer demographics**, including gender, as well as internet subscription details such as subscription plans and payment methods.

After loading the dataset, we will perform simple **data cleaning and feature selection**. 

In the final step, we will split the dataset to **features** and **response** sets.

In [0]:
from pyspark.sql.functions import col

# dataset path
dataset_p_telco = f"{DA.paths.datasets}/telco/telco-customer-churn.csv"

# features to use
primary_key = "customerID"
response = "Churn"
features = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"] # Keeping numerical only for simplicity and demo purposes

# Read dataset (and drop nan)
telco_df = spark.read.csv(dataset_p_telco, inferSchema=True, header=True, multiLine=True, escape='"')\
            .withColumn("TotalCharges", col("TotalCharges").cast('double'))\
            .withColumn("SeniorCitizen", col("SeniorCitizen").cast('double'))\
            .withColumn("Tenure", col("tenure").cast('double'))\
            .na.drop(how='any')

# Split with 80 percent of the data in train_df and 20 percent of the data in test_df
train_df, test_df = telco_df.randomSplit([.8, .2], seed=42)

# Separate features and ground-truth
features_df = train_df.select(primary_key, *features)
response_df = train_df.select(primary_key, response)

# review the features dataset
display(features_df)

customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0002-ORFBO,0.0,9.0,65.6,593.3
0003-MKNFE,0.0,9.0,59.9,542.4
0011-IGKFF,1.0,13.0,98.0,1237.85
0013-EXCHZ,1.0,3.0,83.9,267.4
0013-MHZWF,0.0,9.0,69.4,571.45
0014-BMAQU,0.0,63.0,84.65,5377.8
0016-QLJIS,0.0,65.0,90.45,5957.9
0017-DINOC,0.0,54.0,45.2,2460.55
0017-IUDMW,0.0,72.0,116.8,8456.75
0018-NYROU,0.0,5.0,68.95,351.5



## Batch Deployment - Without Feature Store

This demo will cover two main batch deployment methods. The first method is deploying models without a feature table. For the second method, we will use a feature table to train the model and later use the feature table for inference.


### Setup Model Registry with UC

Before we start model deployment, we need to fit and register a model. In this demo, **we will log models to Unity Catalog**, which means first we need to setup the **MLflow Model Registry URI**.

In [0]:
import mlflow

# Point to UC model registry
mlflow.set_registry_uri("databricks-uc")
client = mlflow.MlflowClient()

# helper function that we will use for getting latest version of a model
def get_latest_model_version(model_name):
    """Helper function to get latest model version"""
    model_version_infos = client.search_model_versions("name = '%s'" % model_name)
    return max([model_version_info.version for model_version_info in model_version_infos])

### Fit and Register a Model with UC

In [0]:
# Train a sklearn Decision Tree Classification model
from sklearn.tree import DecisionTreeClassifier
from mlflow.models import infer_signature

# Covert data to pandas dataframes
X_train_pdf = features_df.drop(primary_key).toPandas()
Y_train_pdf = response_df.drop(primary_key).toPandas()
clf = DecisionTreeClassifier(max_depth=3, random_state=42)

# Use 3-level namespace for model name
model_name = f"{DA.catalog_name}.{DA.schema_name}.ml_model" 

with mlflow.start_run(run_name="Model-Batch-Deployment-Demo") as mlflow_run:

    # Enable automatic logging of input samples, metrics, parameters, and models
    mlflow.sklearn.autolog(
        log_input_examples=True,
        log_models=False,
        log_post_training_metrics=True,
        silent=True)
    
    clf.fit(X_train_pdf, Y_train_pdf)

    # Log model and push to registry
    signature = infer_signature(X_train_pdf, Y_train_pdf)
    mlflow.sklearn.log_model(
        clf,
        artifact_path="decision_tree",
        signature=signature,
        registered_model_name=model_name
    )

    # Set model alias (i.e. Baseline)
    client.set_registered_model_alias(model_name, "Baseline", get_latest_model_version(model_name))

Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Successfully registered model 'labuser8891327_1736105957_wp0t_da.default.ml_model'.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '1' of model 'labuser8891327_1736105957_wp0t_da.default.ml_model'.
2025/01/05 19:56:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model-Batch-Deployment-Demo at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208254/runs/adf8bae3f85b448281f5bafbce69655b.
2025/01/05 19:56:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208254.



### Use the Model for Inference 

Now that our model is ready in model registry, we can use it for inference. In this section we will use the model for inference directly on a spark dataframe, which called **batch inference**.

#### Load the Model

Loading a model from UC-based model registry is done by getting a model using **alias** and **version**. 

After loading the model, we will create a **`spark_udf`** from the model.

In [0]:
latest_model_version = client.get_model_version_by_alias(name=model_name, alias="baseline").version
model_uri = f"models:/{model_name}/{latest_model_version}" # Should be version 1
# model_uri = f"models:/{model_name}@baseline # uri can also point to @alias
predict_func = mlflow.pyfunc.spark_udf(
    spark,
    model_uri)

2025/01/05 20:03:43 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2025/01/05 20:03:44 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


#### Inference

Next, we will simply use the created function for inference.

In [0]:
# prepare test dataset
test_features_df = test_df.select(primary_key, *features)

# make prediction
prediction_df = test_features_df.withColumn("prediction", predict_func(*test_features_df.drop(primary_key).columns))

display(prediction_df)

customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,prediction
0004-TLHLJ,0.0,4.0,73.9,280.85,Yes
0013-SMEOE,1.0,71.0,109.7,7904.25,No
0015-UOCOJ,1.0,7.0,48.2,340.35,No
0019-EFAEP,0.0,72.0,101.3,7261.25,No
0023-HGHWL,1.0,1.0,25.1,25.1,No
0030-FNXPP,0.0,3.0,19.85,57.2,No
0042-RLHYP,0.0,69.0,19.7,1396.9,No
0057-QBUQH,0.0,43.0,25.1,1070.15,No
0078-XZMHT,0.0,72.0,85.15,6316.2,No
0080-EMYVY,0.0,14.0,51.45,727.85,No


## Batch Deployment - With Feature Store 

In the previous section we trained and registered a model using Spark dataframe. In some cases, you will need to use features from a feature store for training and inference. 

In this section we will demonstrate how to train and deploy a model using Feature Store.


### Create Feature Table

Let's create a feature table based on the `features_df` that we created before. Please note that we will be using **Feature Store with Unity Catalog**, which means we need to use **`FeatureEngineeringClient`**.

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient

# prepare feature set
features_df_all = telco_df.select(primary_key, *features)

# feature table definition
fe = FeatureEngineeringClient()
feature_table_name = f"{DA.catalog_name}.{DA.schema_name}.features"

#drop table if exists
try:
    fe.drop_table(name=feature_table_name)
except:
    pass

# Create feature table
fe.create_table(
    name=feature_table_name,
    df=features_df_all,
    primary_keys=[primary_key],
    description="Example feature table"
)

2025/01/05 20:13:06 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['customerID'] of table 'labuser8891327_1736105957_wp0t_da.default.features' to NOT NULL.
2025/01/05 20:13:12 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['customerID'] on table 'labuser8891327_1736105957_wp0t_da.default.features'.
2025/01/05 20:13:30 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'labuser8891327_1736105957_wp0t_da.default.features'.


<FeatureTable: name='labuser8891327_1736105957_wp0t_da.default.features', table_id='13ab077c-0e05-48a3-9f7f-bb8c50db4a2e', description='Example feature table', primary_keys=['customerID'], partition_columns=[], features=['customerID', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], creation_timestamp=1736107985756, online_stores=[], notebook_producers=[], job_producers=[], table_data_sources=[], path_data_sources=[], custom_data_sources=[], timestamp_keys=[], tags={}>

### Setup Feature Lookups

In order to create a training set from the feature table, we need to define a *feature lookup*. This will be used for creating training set from the feature table. 

Note that the **`lookup_key`** is used for matching records in feature table.

In [0]:
# Create training set based on feature lookup
from databricks.feature_engineering import FeatureLookup

fl_handle = FeatureLookup(
    table_name=feature_table_name,
    lookup_key=[primary_key]
)

training_set_spec = fe.create_training_set(
    df=response_df,
    label=response,
    feature_lookups=[fl_handle],
    exclude_columns=[primary_key]
)

# Load training dataframe based on defined feature-lookup specification
training_df = training_set_spec.load_df()

### Fit and Register a Model with UC using Feature Table

After creating the training set, **model training and registering is the same as the previous step**.

In [0]:
import warnings
from mlflow.types.utils import _infer_schema
    
    
# Convert data to pandas dataframes
X_train_pdf2 = training_df.drop(primary_key, response).toPandas()
Y_train_pdf2 = training_df.select(response).toPandas()
clf2 = DecisionTreeClassifier(max_depth=3, random_state=42)


with mlflow.start_run(run_name="Model-Batch-Deployment-Demo-With-FS") as mlflow_run:

    # Enable automatic logging of input samples, metrics, parameters, and models
    mlflow.sklearn.autolog(
        log_input_examples=True,
        log_models=False,
        log_post_training_metrics=True,
        silent=True)
    
    clf2.fit(X_train_pdf, Y_train_pdf)

    # Infer output schema
    try:
      output_schema = _infer_schema(Y_train_pdf)
    except Exception as e:
      warnings.warn(f"Could not infer model output schema: {e}")
      output_schema = None
    
    # Log using feature engineering client and push to registry
    fe.log_model(
        model=clf2,
        artifact_path="decision_tree",
        flavor=mlflow.sklearn,
        training_set=training_set_spec,
        output_schema=output_schema,
        registered_model_name=model_name
    )

    # Set model alias (i.e. Champion)
    client.set_registered_model_alias(model_name, "Champion", get_latest_model_version(model_name))

Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

/databricks/python/lib/python3.11/site-packages/mlflow/types/schema.py:636: FutureWarning: `optional` is deprecated and will be removed in a future version of MLflow. Use `required` instead.
  warnings.warn(


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Registered model 'labuser8891327_1736105957_wp0t_da.default.ml_model' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Created version '2' of model 'labuser8891327_1736105957_wp0t_da.default.ml_model'.
2025/01/05 20:21:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model-Batch-Deployment-Demo-With-FS at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208254/runs/45662dfd7e55406eb83dca1e2bca7696.
2025/01/05 20:21:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208254.



### Use the Model for Inference

Inference for models that are registered with a Feature Store table is different than inference with Spark dataframe. For inference, we will use **feature engineering client's `.score_batch()` method**. This method takes **a model URI** and **dataframe with primary key info**.

**So how does the function know which feature table to use?** If you visit **Artifacts** section of registered model, you will see a **`data`** folder is registered with the model. Also, model file includes **`data: data/feature_store`** statement to define feature data.


In [0]:
champion_model_uri = f"models:/{model_name}@champion"

In [0]:
# prepare lookup dataset
lookup_df = test_df.select("customerID")

# predict in batch using lookup df
prediction_fe_df = fe.score_batch(
    model_uri=champion_model_uri,
    df=lookup_df,
    result_type='string')

2025/01/05 20:22:53 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2025/01/05 20:22:53 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


In [0]:
display(prediction_fe_df)

customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,prediction
0004-TLHLJ,0.0,4.0,73.9,280.85,Yes
0013-SMEOE,1.0,71.0,109.7,7904.25,No
0015-UOCOJ,1.0,7.0,48.2,340.35,No
0019-EFAEP,0.0,72.0,101.3,7261.25,No
0023-HGHWL,1.0,1.0,25.1,25.1,No
0030-FNXPP,0.0,3.0,19.85,57.2,No
0042-RLHYP,0.0,69.0,19.7,1396.9,No
0057-QBUQH,0.0,43.0,25.1,1070.15,No
0078-XZMHT,0.0,72.0,85.15,6316.2,No
0080-EMYVY,0.0,14.0,51.45,727.85,No


## Performance Considerations

There are many possible (write) optimizations that Delta Lake can offer such as:
- Partitioning: stores data associated with different categorical values in different directories.
- Z-Ordering: colocates related information in the same set of files.
- **Liquid Clustering:** replaces both above-mentioned  methods to simplify data layout decisions and optimize query performance.
- **Predictive Optimizations:** removes the need to manually manage maintenance operations for Delta tables on Databricks.

In this demo, we will show the last two options; liquid clustering and predictive optimization.

In [0]:
spark.sql(f"USE CATALOG {DA.catalog_name}")
spark.sql(f"USE SCHEMA {DA.schema_name}")

DataFrame[]

**Enable Predictive Optimization** at schema level (can also be done at catalog level)

In [0]:
spark.sql(f"ALTER SCHEMA {DA.catalog_name}.{DA.schema_name} ENABLE PREDICTIVE OPTIMIZATION;")

DataFrame[]

Create inference table (where batch scoring jobs would materialized) and enable liquid clustering on using `CLUSTER BY`

In [0]:
%sql
CREATE OR REPLACE TABLE batch_inference(
  customerID STRING
 ,Churn STRING
 ,SeniorCitizen DOUBLE
 ,tenure DOUBLE
 ,MonthlyCharges DOUBLE
 ,TotalCharges DOUBLE
 ,prediction STRING)
CLUSTER BY (customerID, tenure)

In [0]:
(
  prediction_fe_df.write
  .mode("append")
  .option("mergeSchema", True)
  .saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.batch_inference")
)

Manually optimize table

In [0]:
%sql
ANALYZE TABLE batch_inference COMPUTE STATISTICS FOR ALL COLUMNS;
OPTIMIZE batch_inference

path,metrics
s3://unity-catalogs-us-west-2/3b3f54a9-8479-4876-991f-81ec9aee0ea3/tables/ed98ce1f-5a41-4958-a9fa-bfc89522c86b,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 0, false, 0, 0, 1736108791849, 1736108798571, 4, 0, null, List(0, 0), 7, 7, 0, 0, List(31265, true, false, 0, 0, 0, 0, 1, 31265, 31265, null, log, 16777216, 67108864, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(252, 150, 0, 0, 0, 1253), 2, 1, 5, sizeAware))"



## Clean up Classroom

Run the following cell to remove lessons-specific assets created during this lesson.

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the catalog "labuser8891327_1736105957_wp0t_da"...(1 seconds)

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)



## Conclusion

In this demo, we presented two main batch deployment methods using MLflow for model tracking and logging with Unity Catalog. In the first approach, we trained and registered a model without a feature table, reloading it for inference through a Spark UDF. The second method involved training a model with a feature table, registering it in the model registry, and using a look-up key for data retrieval during inference.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>